# Imports

In [1]:
import pandas as pd
import numpy as np

---

In [2]:
# Columnas a cargar
columnas_cargar = ["date", "event_id","ref_type","ref_hash","application_id","attributed", "session_user_agent", "kind", "wifi"]
# Definicion de tipos
tipos = {"event_id" : "int16",
         "ref_hash" : "int64",
         "application_id" : "int16",
         "attributed": "bool",
         "wifi":"bool"}

events = pd.read_csv("../../../../data/tp2/events.csv", usecols=columnas_cargar, dtype=tipos, parse_dates=["date"])


### Formateo el sistema operativo

In [3]:
events['os'] = 1 * ((events["ref_type"] == 1891515180541284343).astype('int8'))

In [4]:
events.drop(columns = {"ref_type"},inplace = True)

In [5]:
events["os"].value_counts()

1    6421584
0    1322997
Name: os, dtype: int64

### 1 ---> ANDROID 
### 0 ---> IOS

In [6]:
events.sort_values(by = "date",inplace = True)

In [7]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

In [8]:
def armar_ventanas(dataset,col_fecha):
    """Recibe el dataset y una string indicando el 
    nombre de la columna que lleva la fecha"""
    
    ventanas = []
    for x in range(5):
        ventanas.append((dataset.loc[((18+x) <= dataset[col_fecha].dt.day) & (dataset[col_fecha].dt.day <= (20+x))]))
    return ventanas

In [9]:
def target_ventanas(ventanas):
    entrenamientos = []
    for ventana in ventanas:
        ent = ventana["ref_hash"].drop_duplicates().to_frame()
        ent.rename(columns ={"ref_hash":"device_id"},inplace = True)
        ent.set_index("device_id", inplace=True)
        entrenamientos.append(ent)
    return entrenamientos

# Armado de ventanas: 

In [10]:
ventanas = armar_ventanas(events,"date")

In [11]:
targets = target_ventanas(ventanas)

In [12]:
def calcular_features(ventanas,feature):
    features = []
    for x in range(5):
        features.append(feature(ventanas[x]))
    return features

In [13]:
def mergear_features(targets,features):
    for x in range(5):
        targets[x] = targets[x].merge(features[x], how = 'left',left_index = True,right_index = True)


## Promedio de eventos por día

In [14]:
def prom_eventos(ventana):
        
        feature = ventana.groupby(["ref_hash","dayofweek"]).size()\
                         .groupby("ref_hash").mean()\
                         .to_frame().rename(columns = {0:"promedio_eventos"})
        feature["promedio_eventos"] = feature["promedio_eventos"].astype("int64")
        return feature

In [16]:
features = calcular_features(ventanas,prom_eventos)

In [17]:
mergear_features(targets,features)

In [18]:
entrenamiento1 = targets[0]

In [19]:
entrenamiento1.head()

,promedio_eventos
device_id,
4314242242878368557,3
2068670382964280534,70
726218951290512705,27
1329210878469536477,52
5661732594213028190,16


## Evento más frecuente (ESTO ES VIEJO, LO ACTUALICÉ ABAJO, LO DEJO COMO ANÁLISIS)

In [20]:
ventana1 = ventanas[0]
feature = ventana1.groupby(["ref_hash","event_id"]).size().to_frame()
idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
feature = feature[idx]

In [21]:
feature.rename(columns= {0:"cant_ocurrencias"},inplace = True)

In [22]:
feature.head()

,,cant_ocurrencias
ref_hash,event_id,
41863526108385,1,52
161514654074162,23,4
186034136943920,1,10
360710529886978,23,2
365882020742330,15,41


#### ¿Hay dispositivos que tengan más de un evento como máximo ?

In [23]:
((feature.groupby(["ref_hash"]).count()["cant_ocurrencias"]) > 1).sum()

27801

#### ¿Cual es el máximo numero de eventos empatados como máximo por dispositivo?

In [24]:
(feature.groupby(["ref_hash"]).count()).max()

cant_ocurrencias    10
dtype: int64

##### ¿Qué criterio se toma para decidir con cual evento quedarse? 

##### Voy a crear una lista ordenada de los eventos más presentes por dispositivo, contando un solo dispositivo por evento para no desbalancear si un dispositivo hace 10000 el mismo evento. Esto podría dar un orden de importancia entre eventos

In [25]:
top_eventos = ventana1.groupby(["ref_hash","event_id"]).size().reset_index()\
                      .groupby("event_id").size().sort_values(ascending = False)

In [26]:
top_eventos.head()

event_id
23    63062
15    45078
7     32720
0     25747
2     20475
dtype: int64

In [27]:
top_eventos = top_eventos.index.to_list()

### Cantidad de veces que se realizó el evento más frecuente

In [28]:
def prom_eventos(ventana):
        
        feature = ventana.groupby(["ref_hash","dayofweek"]).size()\
                         .groupby("ref_hash").mean()\
                         .to_frame().rename(columns = {0:"promedio_eventos"})
        feature["promedio_eventos"] = feature["promedio_eventos"].astype("int64")
        return feature

In [29]:
def maximo_evento(ventana):
    
    feature = ventana.groupby(["ref_hash","event_id"]).size().to_frame()
    idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
    feature = feature[idx]
    feature.rename(columns= {0:"max_cant_event"},inplace = True)
    feature.reset_index(inplace = True)
    feature = feature[["ref_hash","max_cant_event"]].drop_duplicates("ref_hash")
    feature = feature.set_index("ref_hash")
    return feature    

In [30]:
features = calcular_features(ventanas,maximo_evento)

In [31]:
mergear_features(targets,features)

In [32]:
entrenamiento1 = targets[0]

In [33]:
entrenamiento1.head()

,promedio_eventos,max_cant_event
device_id,,
4314242242878368557,3,5
2068670382964280534,70,36
726218951290512705,27,16
1329210878469536477,52,65
5661732594213028190,16,23


## Cuantos eventos comparten la primera posición de evento más frecuente

In [34]:
def cant_eventos_maximos(ventana):
    feature = ventana.groupby(["ref_hash","event_id"]).size().to_frame()
    idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
    feature = feature[idx]
    feature = feature.groupby("ref_hash")[0].count().to_frame()
    feature.rename(columns= {0:"cant_eventos_maximos"},inplace = True)
    return feature    
    
    

In [35]:
features = calcular_features(ventanas,cant_eventos_maximos)

In [36]:
mergear_features(targets,features)

In [37]:
e1 = targets[0]

In [38]:
e1.head()

,promedio_eventos,max_cant_event,cant_eventos_maximos
device_id,,,
4314242242878368557,3,5,1
2068670382964280534,70,36,1
726218951290512705,27,16,1
1329210878469536477,52,65,1
5661732594213028190,16,23,1


In [41]:
e1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166002 entries, 4314242242878368557 to 8658033545182476382
Data columns (total 3 columns):
promedio_eventos        166002 non-null int64
max_cant_event          166002 non-null int64
cant_eventos_maximos    166002 non-null int64
dtypes: int64(3)
memory usage: 10.1 MB


In [88]:
e2 = targets[3]

In [89]:
e2 = e1.reset_index()

In [90]:
len(e2)

166002

In [91]:
e2["device_id"].isin(val["ref_hash"]).sum()/len(e2)

0.40768183515861256

In [92]:
val["date"].min()

Timestamp('2019-04-23 00:00:00.021000')

In [87]:
val = (events.loc[((23) <= events["date"].dt.day) & (events["date"].dt.day <= (25))])


## Último evento

#### Pruebo primero con una ventana

In [243]:
ventana1 = ventanas[0]

In [244]:
idx = ventana1.groupby(['ref_hash'])["date"].transform(max) == ventana1["date"]
feature = ventana1[idx]

In [245]:
len(feature)

166018

In [247]:
feature["ref_hash"].nunique()

166002

### Hay valores que tienen la misma fecha con el mismo evento, pero distinto kind, por ejemplo: 

In [248]:
feature[feature["ref_hash"] == 2831641632794815413]

,date,event_id,ref_hash,application_id,attributed,session_user_agent,kind,wifi,os,dayofweek
7488551,2019-04-20 06:55:29.193,109,2831641632794815413,116,False,5.783151e+18,1.020653e+18,False,0,5
7488552,2019-04-20 06:55:29.193,108,2831641632794815413,116,False,5.783151e+18,2.376750e+18,False,0,5


#### Solo me importa el evento así que descarto al azar

In [249]:
feature = feature[["ref_hash","date","event_id","application_id"]].drop_duplicates("ref_hash")
feature.rename(columns = {"ref_hash":"device_id","event_id":"ult_ev","date":"fecha_ult_ev","application_id":"ult_app"},inplace = True)
feature.set_index("device_id",inplace = True)

In [250]:
feature.head()

,fecha_ult_ev,ult_ev,ult_app
device_id,,,
6444651396290009624,2019-04-18 00:00:02.052,245,21
3601178220858933758,2019-04-18 00:00:07.957,246,21
3233293871630648142,2019-04-18 00:00:09.612,15,65
4452972490570146531,2019-04-18 00:00:12.303,7,122
5451417563610992673,2019-04-18 00:00:19.597,106,121


In [253]:
def ult_evento(ventana):

    idx = ventana.groupby(['ref_hash'])["date"].transform(max) == ventana["date"]
    feature = ventana[idx]
    feature = feature[["ref_hash","date","event_id","application_id"]].drop_duplicates("ref_hash")
    feature.rename(columns = {"ref_hash":"device_id","event_id":"ult_ev","date":"fecha_ult_ev","application_id":"ult_app"},inplace = True)
    feature.set_index("device_id",inplace = True)
    return feature

In [254]:
features = calcular_features(ventanas,ult_evento)

In [255]:
mergear_features(targets,features)

In [256]:
e1 = targets[0]

In [257]:
e1.head()

,promedio_eventos,max_cant_event,cant_eventos_maximos,fecha_ult_ev,ult_ev,ult_app
device_id,,,,,,
4314242242878368557,3,5,1,2019-04-20 20:03:34.247,15,65
2068670382964280534,70,36,1,2019-04-20 19:41:28.313,15,49
726218951290512705,27,16,1,2019-04-18 13:39:15.767,15,65
1329210878469536477,52,65,1,2019-04-20 23:39:14.726,508,2
5661732594213028190,16,23,1,2019-04-20 19:36:04.032,2,210


# HASTA ACÁ ES LO ÚLTIMO QUE LLEGUE, LO QUE SIGUE SON COSAS VIEJAS QUE TODAVIA NO LAS ADAPTE 

## ¿Cúal es la hora del día con más actividad para cada usuario ? 

In [101]:
ventana1["hora"] = ventana1["date"].dt.hour

/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [105]:
feature = ventana1.groupby(["ref_hash","hora"]).size()
idx = feature.groupby(['ref_hash']).transform(max) == feature
feature = feature[idx]

In [122]:
feature = feature.to_frame().reset_index()

In [124]:
len(feature)

193332

In [125]:
feature["ref_hash"].nunique()

166002

### Hay dispositivos que comparten varias horas, habrá que decidir el criterio de elección

In [126]:
feature.head()

,ref_hash,hora,0
0,41863526108385,5,64
1,161514654074162,17,8
2,186034136943920,16,13
3,360710529886978,18,2
4,365882020742330,18,8


## Aplicación más usada

In [19]:
feature = ventana1.groupby(['ref_hash',"application_id"]).size().to_frame()

In [25]:
feature.head()

0
ref_hash        application_id    
41863526108385  65              21
                121              3
                210             64
161514654074162 121              8
186034136943920 210             13

In [29]:
idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
feature = feature[idx].reset_index()

In [30]:
feature["ref_hash"].nunique()

166002

In [31]:
len(feature)

168104

### Propuesta: Hay valores repetidos, podría quedarme con la hora más cercana a las 00 hs y calcular el tiempo que falta desde t0 a esa hora. Además de indicar la cantidad de eventos registrados a esa hora, y la cantidad de horas que comparten el primer lugar.

### Propuesta: En vez de indicar valores categoricos como aplicación más usada, evento más registrado, se podría indicar cual es su valor de frecuencia, ejemplo:
#### __Freq_ev_popular__ : 53
#### __Freq_app_popular__ : 12

### Propuesta : En los valores en donde se comparten horas, eventos, aplicaciones, etc, como los más vistos, se podría indicar cual es la cantidad de valores que comparten ese puesto, ejemplo:

#### __cant_horas_mas_activas__: 2
#### __cant_events_en_horas_mas_activas__ : 10